# Implimentation of chatbot using NLP

In [6]:
pip install nltk scikit-learn streamlit

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [4]:
#Importing necessary libraries
import nltk
import random
import os
import ssl
import streamlit as st
import sklearn.feature_extraction.text 
from sklearn.linear_model import LogisticRegression


In [5]:
ssl._create_default_https_context = ssl._create_unverified_context
nltk.data.path.append(os.path.abspath('nltk_data'))
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [ ]:
intents = [
    {
        'tag':'greeting',
        'patterns':['Hi', 'Hello', 'Hey', 'Whats up', 'How are you'],
        'responses':['Hi there', 'Hello', 'Hey', 'Nothing much', 'Im fine, thank you']
    },
    {
        'tag':'goodbye',
        'patterns':['Bye', 'See you later', 'Goodbye', 'Take care'],
        'responses':['Goodbye', 'See you later', 'Take care']
    },
    {
        "tag": "thanks",
        "patterns": ["Thank you", "Thanks", "Thanks a lot", "I appreciate it"],
        "responses": ["You're welcome", "No problem", "Glad I could help"]
    },
    {
        "tag": "about",
        "patterns": ["What can you do", "Who are you", "What are you", "What is your purpose"],
        "responses": ["I am a chatbot", "My purpose is to assist you", "I can answer questions and provide assistance"]
    },
    {
        "tag": "help",
        "patterns": ["Help", "I need help", "Can you help me", "What should I do"],
        "responses": ["Sure, what do you need help with?", "I'm here to help. What's the problem?", "How can I assist you?"]
    },
    {
        "tag": "age",
        "patterns": ["How old are you", "What's your age"],
        "responses": ["I don't have an age. I'm a chatbot.", "I was just born in the digital world.", "Age is just a number for me."]
    },
    {
        "tag": "weather",
        "patterns": ["What's the weather like", "How's the weather today"],
        "responses": ["I'm sorry, I cannot provide real-time weather information.", "You can check the weather on a weather app or website."]
    },
    {
        "tag": "budget",
        "patterns": ["How can I make a budget", "What's a good budgeting strategy", "How do I create a budget"],
        "responses": ["To make a budget, start by tracking your income and expenses. Then, allocate your income towards essential expenses like rent, food, and bills. Next, allocate some of your income towards savings and debt repayment. Finally, allocate the remainder of your income towards discretionary expenses like entertainment and hobbies.", "A good budgeting strategy is to use the 50/30/20 rule. This means allocating 50% of your income towards essential expenses, 30% towards discretionary expenses, and 20% towards savings and debt repayment.", "To create a budget, start by setting financial goals for yourself. Then, track your income and expenses for a few months to get a sense of where your money is going. Next, create a budget by allocating your income towards essential expenses, savings and debt repayment, and discretionary expenses."]
    },
    {
        "tag": "credit_score",
        "patterns": ["What is a credit score", "How do I check my credit score", "How can I improve my credit score"],
        "responses": ["A credit score is a number that represents your creditworthiness. It is based on your credit history and is used by lenders to determine whether or not to lend you money. The higher your credit score, the more likely you are to be approved for credit.", "You can check your credit score for free on several websites such as Credit Karma and Credit Sesame."]
    },
    {
        "tag": "how_are_you",
        "patterns": ["How are you?", "How's it going?", "What's up?", "How have you been?"],
        "responses": [
            "I'm doing great, thanks for asking!",
            "All good here! How about you?",
            "I'm just a bot, but I'm here to help you!"
        ]
    },
    {
        "tag": "bot_name",
        "patterns": ["What's your name?", "Who are you?", "Do you have a name?", "What should I call you?"],
        "responses": [
            "You can call me ChatBot!",
            "I'm your friendly chatbot!",
            "I don't have a name, but you can call me whatever you like!"
        ]
    },
    {
        "tag": "help",
        "patterns": ["Can you help me?", "I need help", "What can you do?", "How does this work?"],
        "responses": [
            "Of course! I can help you with general questions, provide information, or just chat with you.",
            "I'm here to assist you. What do you need help with?",
            "Let me know how I can help!"
        ]
    },
    {
        "tag": "jokes",
        "patterns": ["Tell me a joke", "Make me laugh", "Do you know any jokes?", "Say something funny"],
        "responses": [
            "Why don't scientists trust atoms? Because they make up everything!",
            "Why did the scarecrow win an award? Because he was outstanding in his field!",
            "What do you call fake spaghetti? An impasta!"
        ]
    },
    {
        "tag": "time",
        "patterns": ["What time is it?", "Can you tell me the time?", "What's the current time?"],
        "responses": [
            "I don't have access to real-time data, but you can check your device's clock!",
            "I can't tell the time, but I hope you're having a great day!"
        ]
    },
    {
        "tag": "fun_facts",
        "patterns": ["Tell me a fun fact", "Do you know any interesting facts?", "Give me a random fact"],
        "responses": [
            "Did you know? Honey never spoils. Archaeologists have found pots of honey in ancient Egyptian tombs that are over 3,000 years old and still edible!",
            "Fun fact: Octopuses have three hearts!",
            "Here's a cool fact: Bananas are berries, but strawberries aren't!"
        ]
    },
    {
        "tag": "compliments",
        "patterns": ["You're awesome", "I like you", "You're smart", "You're funny"],
        "responses": [
            "Aww, thank you! You're awesome too!",
            "You're making me blush! 😊",
            "Thanks! You're pretty cool yourself!"
        ]
    },
    {
        "tag": "unknown",
        "patterns": [],  # Catch-all for unrecognized inputs
        "responses": [
            "I'm sorry, I didn't understand that. Can you rephrase?",
            "I'm still learning! Could you explain that differently?",
            "I'm not sure about that. Can I help with something else?"
        ]
    },
    {
        "tag": "capabilities",
        "patterns": ["What can you do?", "What are your features?", "What do you know?"],
        "responses": [
            "I can chat with you, tell jokes, share fun facts, and help with general questions!",
            "I'm here to keep you company and assist with whatever you need!",
            "I can answer questions, provide information, and have a friendly conversation!"
        ]
    },
    {
        "tag": "apologies",
        "patterns": ["I'm sorry", "My bad", "I apologize"],
        "responses": [
            "No worries at all!",
            "It's okay, don't worry about it!",
            "No problem, we all make mistakes!"
        ]
    },
    {
        "tag": "age",
        "patterns": ["How old are you?", "What's your age?", "When were you created?"],
        "responses": [
            "I'm just a chatbot, so I don't have an age!",
            "I was created recently, so I'm still learning!",
            "Age is just a number for me. I'm here to help you!"
        ]
    }

]
